## 021 データを読み込む

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
uselog = pd.read_csv('./input/100knoks/03/use_log.csv')
print(len(uselog))
uselog.head()

In [ ]:
customer_master = pd.read_csv('./input/100knoks/03/customer_master.csv')
print(len(customer_master))
customer_master.head()

In [ ]:
class_master = pd.read_csv('./input/100knoks/03/class_master.csv')
print(len(class_master))
class_master.head()

In [ ]:
campaign_master = pd.read_csv('./input/100knoks/03/campaign_master.csv')
print(len(campaign_master))
campaign_master.head()

## 022 顧客データの整形

In [ ]:
customer = pd.merge(customer_master, class_master, on='class', how='left')
customer = pd.merge(customer, campaign_master, on='campaign_id', how='left')
customer

In [ ]:
customer.isnull().sum()

## 023 顧客データの基礎集計

In [ ]:
customer.groupby('class_name').count()['customer_id']

In [ ]:
customer.groupby('campaign_name').count()['customer_id']

In [ ]:
customer.groupby('gender').count()['customer_id']

In [ ]:
customer.groupby('is_deleted').count()['customer_id']

In [ ]:
customer['start_date'] = pd.to_datetime(customer['start_date'])
customer_start = customer.loc[customer['start_date'] > pd.to_datetime('2018-4-1')]
print(customer_start)
print(len(customer_start))

## 024 最新顧客データの基礎集計をしてみる

In [ ]:
customer['end_date'] = pd.to_datetime(customer['end_date'])
customer_newer = customer.loc[(customer['end_date'] >= pd.to_datetime('20190331')) | (customer['end_date'].isna())]
print(customer_newer)
print(customer_newer['end_date'].unique())

In [ ]:
customer_newer.groupby('class_name').count()['customer_id']

In [ ]:
customer_newer.groupby('campaign_name').count()['customer_id']

In [ ]:
customer_newer.groupby('gender').count()['customer_id']

## 025 利用履歴データを集計する

In [ ]:
(uselog.isnull()['usedate'] == True).count()
uselog['usedate'] = pd.to_datetime(uselog['usedate'])
uselog['年月'] = uselog['usedate'].dt.strftime("%Y%m")
uselog_months = uselog.groupby(['年月', 'customer_id'], as_index=False).count()
uselog_months.rename(columns={'log_id':'count'}, inplace=True)
del uselog_months['usedate']
uselog_months.head()

In [ ]:
uselog_customer = uselog_months.groupby('customer_id').agg(['mean', 'median', 'max', 'min'])['count']
uselog_customer